# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from skimage.util import crop
from skimage.io import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)
import keras

Using TensorFlow backend.


In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(logical_gpus)
print(tf.__version__)
tf.test.is_gpu_available()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 16 #experiment with the batch size
frame_size = 30//3

In [5]:
print(frame_size)

10


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
from keras.applications.vgg16 import preprocess_input

In [7]:
def generator(source_path, folder_list, batch_size, preprocess=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0,30,30//frame_size) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            if preprocess:
                batch_data = np.zeros((batch_size,frame_size,224,224,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            else:
                batch_data = np.zeros((batch_size,frame_size,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]) #.astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape == (120, 160, 3):
                            image = crop(image,((0,0),(20,20),(0,0)))
                    if preprocess:
                        image = resize(image, (224, 224), anti_aliasing=True)
                        image = preprocess_input(image)
                        batch_data[folder,idx,:,:,0] = image[:,:,0]
                        batch_data[folder,idx,:,:,1] = image[:,:,1]
                        batch_data[folder,idx,:,:,2] = image[:,:,2]
                        
                    else:
                        image = resize(image, (120, 120), anti_aliasing=True)
                        batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        if len(t) %batch_size != 0 :
            rem = len(t)% batch_size
            if preprocess:
                batch_data = np.zeros((rem,frame_size,224,224,3))
            else:
                batch_data = np.zeros((rem,frame_size,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem,5)) # batch_labels is the one hot representation of the output
            imgs = os.listdir(source_path+'/'+ t[-rem].split(';')[0]) # read all the images in the folder
            index = range(0,30,5)
            for folder in range(rem):
                for idx,item in enumerate(index): #  Iterate iver the frames/images of a folder to read them in
                    image   = imread(source_path+'/'+ t[-rem].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if image.shape == (120, 160, 3):
                            image = crop(image,((0,0),(20,20),(0,0)))
                    
                    if preprocess:
                        image = resize(image, (224, 224), anti_aliasing=True)
                        image = preprocess_input(image)
                        batch_data[folder,idx,:,:,0] = image[:,:,0]
                        batch_data[folder,idx,:,:,1] = image[:,:,1]
                        batch_data[folder,idx,:,:,2] = image[:,:,2]
                        
                    else:
                        image = resize(image, (120, 120), anti_aliasing=True)
                        batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[-(rem - folder)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


In [9]:
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, RepeatVector, Input, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

In [11]:
n_classes = 5
features = 224
channels = 3

model = Sequential()
model.add(Conv3D(batch_size, (3, 3, 3), padding='same',
                 input_shape=(frame_size, 120, 120, 3 )))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(batch_size, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.2))

model.add(Conv3D(32, (3, 3, 3), strides=(1,1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(32, (3, 3, 3), strides=(1,1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.2))


model.add(Conv3D(64, (3, 3, 3), strides=(2,2,2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Conv3D(64, (3, 3, 3), strides=(2,2,2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Dropout(0.2))


model.add(Conv3D(128, (3, 3, 3), strides=(4,4,4), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Conv3D(128, (3, 3, 3), strides=(4,4,4), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Dropout(0.2))

model.add(Conv3D(256, (3, 3, 3), strides=(8,8,8), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Conv3D(256, (3, 3, 3), strides=(8,8,8), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 1, 1)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
from keras.optimizers import Adam
optimiser = Adam(lr=0.002) #write your optimizer lr=0.001
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 10, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 10, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 120, 120, 16)  64        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 8, 118, 118, 16)   6928      
_________________________________________________________________
activation_2 (Activation)    (None, 8, 118, 118, 16)   0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 118, 118, 16)   64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 59, 59, 16)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [43]:
if not os.path.exists('models'):
    os.mkdir('models')

model_name = 'models/model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

#checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# ReduceLROnPlateau(monitor='categorical_accuracy', factor= 0.125, patience=3, min_lr=0.001)
LR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor= 0.0625, patience=3, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [16]:
print(steps_per_epoch)
print(validation_steps)

42
7


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [17]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0, use_multiprocessing=False)

Epoch 1/25
42/42 [==============================] - 59s 1s/step - loss: 1.1857 - categorical_accuracy: 0.5671 - val_loss: 2.1355 - val_categorical_accuracy: 0.2100

Epoch 00001: val_categorical_accuracy improved from -inf to 0.21000, saving model to models/model_init_2020-05-0413_16_32.220504/model-00001-1.17429-0.56712-2.13546-0.21000.h5
Epoch 2/30
42/42 [==============================] - 58s 1s/step - loss: 1.1127 - categorical_accuracy: 0.5943 - val_loss: 3.7175 - val_categorical_accuracy: 0.1500

Epoch 00002: val_categorical_accuracy did not improve from 0.21000
Epoch 3/30
42/42 [==============================] - 56s 1s/step - loss: 1.0648 - categorical_accuracy: 0.6290 - val_loss: 8.3055 - val_categorical_accuracy: 0.2300

Epoch 00003: val_categorical_accuracy improved from 0.21000 to 0.23000, saving model to models/model_init_2020-05-0413_16_32.220504/model-00003-1.04317-0.62896-8.30549-0.23000.h5
Epoch 4/30
42/42 [==============================] - 56s 1s/step - loss: 1.0408 - ca